In [ ]:
#from tweepy import OAuthHandler, Stream, StreamListener
from tweepy import OAuthHandler, Stream, StreamListener
import tweepy
import json
from datetime import datetime
import sqlalchemy

In [ ]:
# Credenciais para utilização da API do Twitter

API_Key = ""
API_secret_key = ""

access_token = ""
access_token_secret = ""

In [ ]:
#Realizar autenticação no Twitter
auth = OAuthHandler(API_Key, API_secret_key)
auth.set_access_token(access_token, access_token_secret)


In [ ]:
# Construir uma instancia da API
api = tweepy.API(auth)

In [ ]:
# Definir um aruiqvo para armazenar tweet coletados
data_hoje = datetime.now().strftime("%y-%m-%d-%H-%M-%S")
out = open(f"collected_tweets_{data_hoje}.txt", "w")

In [ ]:
class MyListener(StreamListener):

    def on_data(self, data):
        # print(data)
        itemString = json.dumps(data)
        out.write(itemString + "\n")
        return True

    def on_error(self, status):
        print(status)

In [ ]:
l = MyListener()
stream = Stream(auth, l)
stream.filter(track=["BBB"])

In [ ]:
# Abrir Arquivo de tweets
with open('./collected_tweets_21-03-11-21-18-48.txt', 'r') as file:
    tweets = file.readlines()

In [ ]:
# Fazendo o parse do tweet para um formato json e exporta arquivo
with open('tweet.json', 'w') as f:
    json.dump(
        json.loads(json.loads(tweets[0])), f
    )

In [ ]:
type(json.loads(json.loads(tweets[0])))

In [ ]:
parsed_tweets = [json.loads(json.loads(i)) for i in tweets]

In [ ]:
# Quantidade de tweets coletados
len(parsed_tweets[0])

## Acessando as chaves do JSON (Dicionário)

In [ ]:
primeirotweet = parsed_tweets[0]

In [ ]:
# Acessa o texto do tweet
# Cahve de primeiro Nivel(root)
primeirotweet["text"]

In [ ]:
#Acessar chave user
primeirotweet["user"]["id"]

## Transformar os tweets em, uam estrutura relacional analisável

In [ ]:
import pandas as pd

In [ ]:
teste = pd.DataFrame(primeirotweet)
teste.columns
#Não Funciona!!

In [ ]:
df_tratado = pd.DataFrame(primeirotweet).reset_index(drop=True).iloc[:1]

In [ ]:
df_tratado

In [ ]:
df_tratado.columns

In [ ]:
# Removendo algumas colunas
df_tratado.drop(columns=['quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'user', 'entities', 'retweeted_status'], inplace=True)

In [ ]:
df_tratado.columns

In [ ]:
df_tratado['user_id'] = primeirotweet['user']['id']
df_tratado['user_id_str'] = primeirotweet['user']['id_str']
df_tratado['user_screen_name'] = primeirotweet['user']['screen_name']
df_tratado['user_location'] = primeirotweet['user']['location']
df_tratado['user_description'] = primeirotweet['user']['description']
df_tratado['user_protected'] = primeirotweet['user']['protected']
df_tratado['user_verified'] = primeirotweet['user']['verified']
df_tratado['user_followers_count'] = primeirotweet['user']['followers_count']
df_tratado['user_friends_count'] = primeirotweet['user']['friends_count']
df_tratado['user_created_at'] = primeirotweet['user']['created_at']

In [ ]:
df_tratado

In [ ]:
# Entidades
primeirotweet['entities']

In [ ]:
user_mentions = []

for i in range(len(primeirotweet['entities']['user_mentions'])):
    dicionariobase = primeirotweet['entities']['user_mentions'][i].copy()
    dicionariobase.pop('indices', None)
    df = pd.DataFrame(dicionariobase, index=[0])
    df = df.rename(columns={
        'screen_name': 'entities_screen_name',
        'name': 'entities_name',
        'id': 'entities_id',
        'id_str': 'entities_id_str'
    })
    user_mentions.append(df)

In [ ]:
user_mentions

In [ ]:
pd.concat(user_mentions, ignore_index=True)

In [ ]:
dfs = []
for i in user_mentions:
    dfs.append(
        pd.concat([df_tratado.copy(), i], axis=1)
    )

In [ ]:
pd.concat(dfs, ignore_index=True)

Vamos criar um função que faz todo o tratamento do tweet e converte para um DataFram pandas

In [ ]:
def tweet_para_df(tweet):
    try:
        df_tratado = pd.DataFrame(tweet).reset_index(drop=True).iloc[:1]
        df_tratado.drop(columns=['quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'user', 'entities', 'retweeted_status'], inplace=True)
        df_tratado['user_id'] = tweet['user']['id']
        df_tratado['user_id_str'] = tweet['user']['id_str']
        df_tratado['user_screen_name'] = tweet['user']['screen_name']
        df_tratado['user_location'] = tweet['user']['location']
        df_tratado['user_description'] = tweet['user']['description']
        df_tratado['user_protected'] = tweet['user']['protected']
        df_tratado['user_verified'] = tweet['user']['verified']
        df_tratado['user_followers_count'] = tweet['user']['followers_count']
        df_tratado['user_friends_count'] = tweet['user']['friends_count']
        df_tratado['user_created_at'] = tweet['user']['created_at']

        user_mentions = []

        for i in range(len(tweet['entities']['user_mentions'])):
            dicionariobase = tweet['entities']['user_mentions'][i].copy()
            dicionariobase.pop('indices', None)
            df = pd.DataFrame(dicionariobase, index=[0])
            df = df.rename(columns={
                'screen_name': 'entities_screen_name',
                'name': 'entities_name',
                'id': 'entities_id',
                'id_str': 'entities_id_str'
            })
            user_mentions.append(df)

        dfs = []
        for i in user_mentions:
            dfs.append(
                pd.concat([df_tratado.copy(), i], axis=1)
            )

        df_final = pd.concat(dfs, ignore_index=True)
    except:
        return None
    return df_final

In [ ]:
tweet_para_df(parsed_tweets[5])

In [ ]:
parsed_tweets[2]

In [ ]:
%%time
# Vamos iterar e transformar todos os tweets
parseados = [tweet_para_df(tweet) for tweet in parsed_tweets]

In [ ]:
parseados[5]

In [ ]:
# Eliminar as posições vazias da lista
parseados = [i for i in parseados if i is not None]

In [ ]:
len(parseados)

In [ ]:
tratado = pd.concat(parseados, ignore_index=True)

In [ ]:
tratado

In [ ]:
# Inciando conexão com BD
engine = sqlalchemy.create_engine(
   'mssql+pyodbc://DESKTOP-8C38QEI\SQLEXPRESS/desafio01?driver=SQL Server?Trusted_Connection=yes'
)

In [ ]:
# Inserindo tabela e dados
tratado.to_sql("twitter", engine, index=False, if_exists='replace')